In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from  pyspark.sql.window import Window

spark = SparkSession.builder.appName("ui").getOrCreate()

employees = [
    {"emp_id": 101, "name": "Arun",    "dept": "IT",      "salary": 80000, "experience": 5,  "skills": ["Python", "SQL"],      "status": "Active",   "hire_date": "2021-01-10"},
    {"emp_id": 102, "name": "Meena",   "dept": "HR",      "salary": 55000, "experience": 3,  "skills": ["Excel"],               "status": "Active",   "hire_date": "2022-03-15"},
    {"emp_id": 103, "name": "Kiran",   "dept": "IT",      "salary": 90000, "experience": 7,  "skills": ["Python", "PySpark"],  "status": "Resigned", "hire_date": "2018-10-01"},
    {"emp_id": 104, "name": "Bhavani", "dept": "Finance", "salary": 65000, "experience": 4,  "skills": ["Excel", "SQL"],        "status": "Active",   "hire_date": "2020-07-19"},
    {"emp_id": 105, "name": "Akash",   "dept": "IT",      "salary": 75000, "experience": 2,  "skills": ["Python"],              "status": "Active",   "hire_date": "2023-05-02"},
    
    {"emp_id": 106, "name": "Ravi",    "dept": "Sales",   "salary": 45000, "experience": 1,  "skills": [],                      "status": "Active",   "hire_date": "2024-01-11"},
    {"emp_id": 107, "name": "Divya",   "dept": "HR",      "salary": None,  "experience": 6,  "skills": ["Excel", "PowerBI"],    "status": "Active",   "hire_date": "2019-11-28"},
    {"emp_id": 108, "name": "Suresh",  "dept": "Sales",   "salary": 47000, "experience": 2,  "skills": ["Negotiation"],         "status": "Active",   "hire_date": "2021-06-22"},
    {"emp_id": 109, "name": "Jaya",    "dept": "IT",      "salary": 80000, "experience": 5,  "skills": ["SQL"],                 "status": "Active",   "hire_date": "2020-09-10"},
    {"emp_id": 110, "name": "Vijay",   "dept": "Finance", "salary": 70000, "experience": 8,  "skills": ["Excel"],               "status": "Resigned", "hire_date": "2017-12-01"},

    {"emp_id": 111, "name": "Rohit",   "dept": "IT",      "salary": 65000, "experience": 3,  "skills": ["Python"],              "status": "Active",   "hire_date": "2022-02-17"},
    {"emp_id": 112, "name": "Maya",    "dept": "HR",      "salary": 62000, "experience": 4,  "skills": None,                    "status": "Active",   "hire_date": None},
    {"emp_id": 113, "name": "Goutham", "dept": "Sales",   "salary": 52000, "experience": 3,  "skills": ["Negotiation", "CRM"],  "status": "Active",   "hire_date": "2021-08-05"},
    {"emp_id": 114, "name": "Lavanya", "dept": "IT",      "salary": 90000, "experience": 10, "skills": ["Python", "AWS"],      "status": "Active",   "hire_date": "2015-07-14"},
    {"emp_id": 115, "name": "Kavya",   "dept": "Finance", "salary": 58000, "experience": 1,  "skills": ["Excel"],               "status": "Active",   "hire_date": "2023-01-20"},

    {"emp_id": 116, "name": "Manish",  "dept": "IT",      "salary": None,  "experience": 4,  "skills": ["SQL"],                 "status": "Active",   "hire_date": "2020-10-05"},
    {"emp_id": 117, "name": "Rakesh",  "dept": "Sales",   "salary": 48000, "experience": 2,  "skills": ["CRM"],                 "status": None,       "hire_date": "2022-11-11"},
    {"emp_id": 118, "name": "Anita",   "dept": "HR",      "salary": 56000, "experience": 3,  "skills": ["Excel"],               "status": "Active",   "hire_date": "2023-03-01"},
    {"emp_id": 119, "name": "Tarun",   "dept": "Finance", "salary": 70000, "experience": 7,  "skills": ["SQL", "PowerBI"],      "status": "Active",   "hire_date": "2018-02-25"},
    {"emp_id": 120, "name": "Sanjay",  "dept": "IT",      "salary": 76000, "experience": 5,  "skills": ["Python", "SQL"],       "status": "Active",   "hire_date": "2019-09-09"}
]

print("Spark UI URL:", spark.sparkContext.uiWebUrl)

spark = SparkSession.builder.appName("practice").getOrCreate()

df = spark.createDataFrame(employees)

In [ ]:
# ✅ Question 1
# Write PySpark code to filter only employees from the IT department whose salary is greater than 75,000.
print("Spark UI URL:", spark.sparkContext.uiWebUrl)
df.filter((F.col("dept")=='IT') & (F.col("salary") >75000)).show()

In [ ]:
# Question:
# Write a PySpark query to find all employees who joined in or after the year 2021.

df.filter(F.year(F.col("hire_date"))>=2021).show()

In [ ]:
# Find employees who have Python in their skills.
df.filter(F.array_contains("skills","Python")).select('name','skills').show()

In [ ]:
# Find the highest-paid employee in each department using a window function.
# Return: emp_id, name, dept, salary, rank.
print("Spark UI URL:", spark.sparkContext.uiWebUrl)
win = Window.partitionBy("dept").orderBy(F.col("salary").desc())

df1 = df.withColumn("rank",F.rank().over(win))\
  .withColumn("dense_rank",F.dense_rank().over(win))\
  .filter(F.col("rank")==1).select("emp_id","name","dept","salary","rank").show()

In [ ]:
# Write a PySpark query to explode the skills array so that each skill appears in a separate row.
# Return only:
# emp_id, name, skill

df2 = df.withColumn("skill",F.explode("skills"))
df2.select('emp_id','name','skill').show()

In [ ]:
# Write a PySpark query to replace all null salaries with 0.
df.fillna(0,subset=["salary"])

In [ ]:
# Write a PySpark query to count how many employees are present in each department.
# Return columns: dept, emp_count

df3 = df.groupBy("dept").agg(F.count("name").alias("number_of_emp"))
df3.show()

In [ ]:
# Some employees have status = null.
# Write a PySpark query to replace null status with "Unknown" using when and otherwise.

df4 =df.withColumn("status",F.when(F.col("status").isNull(),"Unknown").otherwise(F.col("status")))
df4.filter(F.col("status")=='Unknown').show()

In [ ]:
# Find the average salary of each department, ignoring null salaries.
# Return: dept, avg_salary

df5 = df.filter(F.col("salary").isNotNull())
df5 =df5.groupBy("dept").agg(F.mean("salary").alias("avg_salary"))
df5.select('dept','avg_salary').show()

In [ ]:
# Task: Find the second highest salary in each department.
# Return: dept, emp_id, name, salary.

wind = Window.partitionBy("dept").orderBy(F.col("salary").desc())

df6 = df.withColumn("rank",F.rank().over(wind))
df6.filter(F.col("rank")==2).select('dept','emp_id','name','salary').show()



In [ ]:
df7 = df.withColumn("skill",F.explode("skills"))
unique_skills_count = df7.select("skill").distinct().count()  # count distinct skills
print(unique_skills_count)

In [ ]:
# Group employees by department and collect all employee names into a list for each department.
# Return columns: dept, emp_list.
df.groupBy("dept").agg(F.collect_list('name')).show(truncate=False)


In [ ]:
from pyspark.sql import functions as F

df_fixed = df.withColumn(
    "skills",
    F.when(F.col("skills").isNull(), F.array()).otherwise(F.col("skills"))
)

df_exploded = df_fixed.withColumn("skill", F.explode("skills"))

df_exploded.select("emp_id", "name", "skill").show(truncate=False)


In [ ]:
df = df.withColumn('hire_date',F.to_date(F.col("hire_date"),'yyyy-MM-dd'))

df = df.withColumn("days_with_company",F.date_diff(F.current_date(),'hire_date'))

df.select('emp_id', 'name', 'hire_date', 'days_with_company').show()

In [ ]:
# Find employees whose name starts with 'A' or ends with 'y'.
# Return: emp_id, name, dept.

df.filter((F.col('name').startswith('A')) | (F.col('name').endswith('y')))\
  .select('emp_id','name','dept').show()

In [ ]:
# Task: Find all duplicate employee names and the count of how many times each name appears.
# Return columns: name, count.

from pyspark.sql import functions as F

df.groupBy("name") \
  .agg(F.count("*").alias("count")) \
  .filter(F.col("count") > 1) \
  .show(truncate=False)


In [ ]:
# Sort employees first by department ascending, then salary descending.
# Return columns: emp_id, name, dept, salary.

df1 =df.sort((['dept','salary']),ascending=[1,0])
df1.select('emp_id','name','dept','salary').show(5)

df1 = df.orderBy(F.col("dept").asc(), F.col("salary").desc())
df1.select('emp_id','name','dept','salary').show(5)


In [ ]:
# Find the top 2 highest-paid employees in each department.
# Return columns: dept, emp_id, name, salary.


w = Window.partitionBy("dept").orderBy(F.col("salary").desc())

df.withColumn('rank',F.rank().over(w))\
  .filter(F.col("rank")<=2).select("dept", "emp_id", "name", "salary").show()

In [ ]:
# Replace null salaries with the average salary of their department.
# Return columns: emp_id, name, dept, salary.
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Define window partitioned by department
win = Window.partitionBy("dept")

# Replace null salaries with average salary of the department
df_filled = df.withColumn(
    "salary",
    F.when(
        F.col("salary").isNull(),
        F.round(F.avg("salary").over(win), 2)  # round to 2 decimals (optional)
    ).otherwise(F.col("salary"))
)

# df_filled.select("emp_id", "name", "dept", "salary").show()




from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Define window to get dept-wise average salary
win = Window.partitionBy("dept")

df_filled = df.withColumn(
    "salary",
    F.coalesce(F.col("salary"), F.round(F.avg("salary").over(win),2))
)

df_filled.select("emp_id", "name", "dept", "salary").show()


In [ ]:
df = df.withColumn('skill',F.explode('skills'))
df.groupBy('skill').agg(F.count('name').alias('emp_count')).show()

In [ ]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

sales_data = [
    {"txn_id": 1, "cust_id": 101, "amount": 500,  "status": "success", "txn_date": "2024-01-01"},
    {"txn_id": 2, "cust_id": 102, "amount": None, "status": "success", "txn_date": "2024-01-01"},
    {"txn_id": 3, "cust_id": 101, "amount": 200,  "status": "failed",  "txn_date": "2024-01-03"},
    {"txn_id": 4, "cust_id": 103, "amount": 800,  "status": "success", "txn_date": "2024-01-04"},
    {"txn_id": 5, "cust_id": 104, "amount": None, "status": "failed",  "txn_date": "2024-01-05"}
]

df = spark.createDataFrame(sales_data)

# Replace null amount with 0
df = df.fillna(0,subset=['amount'])

# Convert txn_date into proper date type
df = df.withColumn('txn_date',to_date(col('txn_date'),'yyyy-MM-dd'))
# df = df.withColumn('hire_date',F.to_date(F.col("hire_date"),'yyyy-MM-dd'))

# Add a new column is_success →
# 1 if status = "success"
#  ->0 otherwise
df = df.withColumn("is_success",when(col('status')=='success',1).otherwise(0))

# Filter only transactions where amount > 0
df = df.filter(col('amount')>0)

# Final output columns:
# txn_id, cust_id, amount, txn_date, is_success
df.select('txn_id', 'cust_id', 'amount', 'txn_date', 'is_success').show()


In [ ]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.window import Window

order_data = [
    {"order_id": 1, "cust_id": 101, "amount": 500, "category": "Electronics", "order_date": "2024-01-01"},
    {"order_id": 2, "cust_id": 101, "amount": 300, "category": "Grocery",     "order_date": "2024-01-03"},
    {"order_id": 3, "cust_id": 102, "amount": 900, "category": "Electronics", "order_date": "2024-01-05"},
    {"order_id": 4, "cust_id": 103, "amount": None,"category": "Grocery",     "order_date": "2024-01-06"},
    {"order_id": 5, "cust_id": 103, "amount": 600, "category": "Fashion",     "order_date": "2024-01-07"},
    {"order_id": 6, "cust_id": 101, "amount": 200, "category": "Fashion",     "order_date": "2024-01-09"}
]

spark = SparkSession.builder.appName("interview").getOrCreate()

df = spark.createDataFrame(order_data)

# 1️⃣ Replace null amount with the average amount of that category

win = Window.partitionBy('category')
df = df.withColumn("amount",coalesce('amount',avg('amount').over(win)))
df.select('order_id', 'cust_id', 'amount', 'category', 'order_date').show()


# 2️⃣ Find total amount per customer
df.groupBy('cust_id').agg(sum('amount').alias("total amount")).show()

# 3️⃣ For each category, find the highest amount order
win = Window.partitionBy('category').orderBy(col('amount').desc())

df.withColumn("dense_rank",dense_rank().over(win))\
  .filter(col("dense_rank")==1)\
  .select('category',"amount").show()

# 4️⃣ Add a new column days_since_order using current_date
df.withColumn("days_since_order",current_date()).show()



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create Spark Session
spark = SparkSession.builder \
    .appName("Join Example - PySpark") \
    .master("local[*]") \
    .getOrCreate()

print(f"spark-ui:,{spark.sparkContext.uiWebUrl}")
# Create first DataFrame: Employees
employees = spark.createDataFrame(
    [
        (1, "John", 1000),
        (2, "Alice", 1200),
        (3, "Bob", 1500)
    ],
    ["emp_id", "name", "salary"]
)

# Create second DataFrame: Departments
departments = spark.createDataFrame(
    [
        (1, "HR"),
        (2, "Engineering"),
        (4, "Marketing")
    ],
    ["emp_id", "department"]
)

# Perform inner join on emp_id
joined_df = employees.join(departments, on="emp_id", how="inner")

# Show result
joined_df.show()


In [ ]:
print("KPMG")
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("CreateDF").getOrCreate()

data = [
    ("Alice", "Badminton, Tennis"),
    ("Bob", "Tennis, Cricket"),
    ("Julie", "Cricket, Carroms")
]

columns = ["name", "hobbies"]

df = spark.createDataFrame(data, columns)

df1 = df.withColumn("Hobbies",explode(split(col("hobbies"),',')))
df1.show()

df2 = df1.groupBy('name').agg(collect_list('Hobbies').alias("new_hobbies"))
df2.withColumn(
    "new_hobbies_str",
    concat(lit('"'),concat_ws(", ", "new_hobbies"),lit('"'))
).show(truncate=False)
# df2.show()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, when

spark = SparkSession.builder.getOrCreate()

# Step 1: Create DataFrame
data = [
    ("Goa", "", "AP"),
    ("", "AP", None),
    (None, "", "bglr")
]

columns = ["city1", "city2", "city3"]

df = spark.createDataFrame(data, columns)

print("Original DataFrame")
df.show()

# Step 2: Create cities column
df = df.withColumn(
    "cities",
    coalesce(
        when(col("city1") != "", col("city1")),
        when(col("city2") != "", col("city2")),
        when(col("city3") != "", col("city3"))
    )
)

print("Final DataFrame")
df.show()


In [ ]:
try:
    spark.stop()
except Exception as e:
    print(e)
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

# Student table
student_data = [
    (1, "Steve"),
    (2, "David"),
    (3, "Aryan")
]

student_columns = ["student_id", "student_name"]

df_students = spark.createDataFrame(student_data, student_columns)

# Marks table
marks_data = [
    (1, "pyspark", 90),
    (1, "sql", 100),
    (2, "sql", 70),
    (2, "pyspark", 60),
    (3, "sql", 30),
    (3, "pyspark", 20)
]

marks_columns = ["student_id", "subject_name", "marks"]

df_marks = spark.createDataFrame(marks_data, marks_columns)

df_marks = df_marks.groupBy('student_id').agg(((sum('marks')/200)*100).alias('marks'))

df_join = df_students.join(df_marks,how='left',on='student_id')
df_join=df_join.withColumn('marks',col('marks').cast('int'))
df_join.withColumn("Result",when(col('marks')>=70,'Distinction')\
                            .when(col('marks').between(60,69),'first class')\
                            .when(col('marks').between(50,59),'second class')\
                            .when(col('marks').between(40,49),'third class')
                            .otherwise('Fail')).show()

In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC Write a solution to find the percentage of immediate orders in the first order of all customers, rounded to 2 decimal places.
# MAGIC --------------------

# COMMAND ----------

# MAGIC %md
# MAGIC If the customer's preferred delivery date is the same as the order date, then the order is called immediate; otherwise, it is called scheduled.
# MAGIC
# MAGIC The first order of a customer is the order with the earliest order date that the customer made. It is guaranteed that a customer has precisely one first order.

# COMMAND ----------

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# COMMAND ----------

spark = SparkSession.builder.appName("test").getOrCreate()
print(f"spark url :,{spark.sparkContext.uiWebUrl}")
# Define the schema
schema = StructType([
    StructField("delivery_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("order_date", StringType(), True),
    StructField("customer_pref_delivery_date", StringType(), True)
])

# Define the data
data = [
    (1, 1, "2019-08-01", "2019-08-02"),
    (2, 2, "2019-08-02", "2019-08-02"),
    (3, 1, "2019-08-11", "2019-08-12"),
    (4, 3, "2019-08-24", "2019-08-24"),
    (5, 3, "2019-08-21", "2019-08-22"),
    (6, 2, "2019-08-11", "2019-08-13"),
    (7, 4, "2019-08-09", "2019-08-09")
]

# Create DataFrame
df = spark.createDataFrame(data, schema=schema)

# Show the DataFrame
df.show(truncate=False)

# COMMAND ----------

# Find the first order for all the customers
first_order = df.groupBy(col("customer_id")).agg(min(col("order_date")).alias("first_order_date"))
first_order.show(truncate=False)

# COMMAND ----------

# Join the first_order date with the original customer order data
first_order_joined = df.join(first_order, df["customer_id"]==first_order["customer_id"], "inner").select(df["*"], first_order["first_order_date"])
first_order_joined.show(truncate=False)

# COMMAND ----------


# Filter immediate deliveries
immediate_deliveries = first_order_joined.filter(col("first_order_date") == col("customer_pref_delivery_date"))
immediate_deliveries.show(truncate=False)

# COMMAND ----------

# Calculate total orders and immediate orders
total_orders = df.count()
immediate_orders = immediate_deliveries.count()
print("immediate_orders: ", immediate_orders, ", total_orders: ", total_orders)

# COMMAND ----------

# Calculate percentage
percentage = (immediate_orders / total_orders) * 100 if total_orders > 0 else 0

# Print the result
print(f"Percentage of immediate orders: {percentage:.2f}%")

# COMMAND ----------



# COMMAND ----------

# MAGIC %md
# MAGIC Using DF API
# MAGIC ----------------

# COMMAND ----------

# Calculate immediate deliveries
df_with_immediate = first_order_joined\
    .withColumn("is_immediate", when(col("first_order_date") == col("customer_pref_delivery_date"), 1).otherwise(0))

# Aggregate to find the percentage of immediate deliveries
result_df = df_with_immediate.agg(round((sum(col("is_immediate")) / count(lit(1)) * 100), 2).alias("immediate_order_percentage"))

# Show the result
result_df.show(truncate=False)


# COMMAND ----------



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

spark = SparkSession.builder.appName("HybridEmployeeData").getOrCreate()

schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("emp_name", StringType(), True),
    StructField("date", StringType(), True),
    StructField("week_day", StringType(), True),
    StructField("first_login", StringType(), True),
    StructField("last_logout", StringType(), True)
])

data = [
    # Week 1
    (101, "Akash", "2024-01-01", "Monday",    "09:05", "18:10"),
    (101, "Akash", "2024-01-02", "Tuesday",   "09:00", "18:00"),
    (101, "Akash", "2024-01-03", "Wednesday", "09:10", "18:20"),
    (101, "Akash", "2024-01-04", "Thursday",  "09:00", "18:05"),
    (101, "Akash", "2024-01-05", "Friday",    "09:15", "17:50"),

    # Week 2
    (101, "Akash", "2024-01-08", "Monday",    "09:00", "18:00"),
    (101, "Akash", "2024-01-09", "Tuesday",   "09:05", "18:10"),
    (101, "Akash", "2024-01-10", "Wednesday", "09:00", "18:00"),
    (101, "Akash", "2024-01-11", "Thursday",  "09:20", "18:30"),
    (101, "Akash", "2024-01-12", "Friday",    "09:10", "17:45"),

    # Week 3
    (101, "Akash", "2024-01-15", "Monday",    "09:00", "18:00"),
    (101, "Akash", "2024-01-16", "Tuesday",   "09:10", "18:15"),
    (101, "Akash", "2024-01-17", "Wednesday", "09:05", "18:05"),
    (101, "Akash", "2024-01-18", "Thursday",  "09:00", "18:00"),
    (101, "Akash", "2024-01-19", "Friday",    "09:20", "17:55"),

    # Week 4
    (101, "Akash", "2024-01-22", "Monday",    "09:05", "18:10"),
    (101, "Akash", "2024-01-23", "Tuesday",   "09:00", "18:00"),
    (101, "Akash", "2024-01-24", "Wednesday", "09:15", "18:25"),
    (101, "Akash", "2024-01-25", "Thursday",  "09:00", "18:05"),
    (101, "Akash", "2024-01-26", "Friday",    "09:10", "17:50")
]

df = spark.createDataFrame(data, schema)
# df.show(truncate=False)


In [ ]:
df = df.withColumn('first_login',date_format(col("first_login"), "HH:mm:ss"))\
       .withColumn('last_logout',date_format(col("first_login"), "HH:mm:ss"))\
       .withColumn('date',to_date(col("date")))

df.printSchema()

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum as _sum, current_date, date_sub
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

# Initialize Spark session
spark = SparkSession.builder.appName("CustomerOrders").getOrCreate()
print(f"spark url :{spark.sparkContext.uiWebUrl}")

# Sample customer data
customer_data = [
    (1, "Alice"),
    (2, "Bob"),
    (3, "Charlie"),
    (4, "David")
]

customer_schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("customer_name", StringType(), True)
])

customers_df = spark.createDataFrame(customer_data, schema=customer_schema)

# Sample orders data
orders_data = [
    (101, 1, "2025-10-10", 500),
    (102, 1, "2025-11-05", 200),
    (103, 2, "2025-09-15", 300),
    (104, 2, "2025-11-20", 150),
    (105, 3, "2025-06-01", 400)  # older than 3 months
]

orders_schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("order_date", StringType(), True),
    StructField("amount", IntegerType(), True)
])

orders_df = spark.createDataFrame(orders_data, schema=orders_schema)
orders_df = orders_df.withColumn("order_date",to_date(col("order_date")))

# Define the date 3 months ago
three_months_ago = date_sub(current_date(), 90)

# Orders in last 3 months
recent_orders_df = orders_df.filter(col("order_date") >= three_months_ago)

# Count orders per customer in last 3 months
orders_count_df = recent_orders_df.groupBy("customer_id").agg(count("order_id").alias("orders_last_3_months"))

# Customers who never made an order in last 3 months
customers_no_orders_df = customers_df.join(orders_count_df, "customer_id", "left") \
                                     .filter(col("orders_last_3_months").isNull()) \
                                     .select("customer_id", "customer_name")

# Customers with orders in last 3 months
customers_with_orders_df = customers_df.join(orders_count_df, "customer_id", "left") \
                                       .na.fill(0)  # Fill 0 for customers with no recent orders

# Show results
print("Customers who never made an order in last 3 months:")
customers_no_orders_df.show()

print("All customers with number of orders in last 3 months:")
customers_with_orders_df.show()


spark url :http://192.168.1.9:4040
Customers who never made an order in last 3 months:
+-----------+-------------+
|customer_id|customer_name|
+-----------+-------------+
|          3|      Charlie|
|          4|        David|
+-----------+-------------+

All customers with number of orders in last 3 months:
+-----------+-------------+--------------------+
|customer_id|customer_name|orders_last_3_months|
+-----------+-------------+--------------------+
|          1|        Alice|                   2|
|          2|          Bob|                   1|
|          3|      Charlie|                   0|
|          4|        David|                   0|
+-----------+-------------+--------------------+

